# vscode中jupyter notebook使用技巧

## cell 命令模式快捷键
注: 进入cell命令模式 只需要按`ESC`即可 命令模式下不区分大小写
- `方向键用来操作选中的单元格`
- `Shift+Enter` : 运行本单元，选中或插入（最后一个Cell的时候）下个单元
- `Ctrl+Enter` : 运行本单元
- `Alt+Enter` : 运行本单元，在其下插入新单元
- `Y` : 单元转入代码状态
- `M`:单元转入markdown状态 （目前尚不支持R 原生状态）
- `Enter` : 转入编辑模式
- `A` : 在上方插入新单元
- `B` : 在下方插入新单元
- `DD` : 删除选中的单元
- `L` : 转换行号
- `Shift+Space` : 向上滚动
- `Space` : 向下滚动


## 引入pytorch框架

In [32]:
import torch

## 张量的创建

In [33]:

x = torch.arange(2,18,2) #创建行向量 从2到18 间隔为2

print(x)

print(x.shape) #获取张量的各个维数

newX = x.reshape((2,2,-1)) #更改x的维数 -1表示按照总元素数 自动推导最后一维的个数

print(newX)

y = torch.zeros((2,3,4)) #创建全零矩阵 维数分别为(2,3,4)

print(y)

z = torch.ones((2,3,4))

print(z)


tensor([ 2,  4,  6,  8, 10, 12, 14, 16])
torch.Size([8])
tensor([[[ 2,  4],
         [ 6,  8]],

        [[10, 12],
         [14, 16]]])
tensor([[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]])
tensor([[[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]],

        [[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]])


## 张量的运算

In [45]:
x = torch.Tensor([1.0,2,4,8])
y = torch.Tensor([2,2,2,2])

# 以下运算相当于对同一位置的元素进行该运算
x + y, x - y, x * y, x / y, x ** y  # **表示求幂运算

# 张量的连接
X = torch.arange(12, dtype=torch.float32).reshape((3,4)) #3行4列
Y = torch.tensor([[2.0, 1, 4, 3], [1, 2, 3, 4], [4, 3, 2, 1]]) #3行4列
torch.cat((X, Y), dim=0),torch.cat((X, Y), dim=1) #按行(第0维)连接 按列(第1维)连接


(tensor([[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.],
         [ 2.,  1.,  4.,  3.],
         [ 1.,  2.,  3.,  4.],
         [ 4.,  3.,  2.,  1.]]),
 tensor([[ 0.,  1.,  2.,  3.,  2.,  1.,  4.,  3.],
         [ 4.,  5.,  6.,  7.,  1.,  2.,  3.,  4.],
         [ 8.,  9., 10., 11.,  4.,  3.,  2.,  1.]]))

In [46]:
# 张量的bool型运算
X==Y # 形成同维bool矩阵 按元素比较

tensor([[False,  True, False,  True],
        [False, False, False, False],
        [False, False, False, False]])

In [50]:
# 广播机制 即对于不同维的张量也是可以进行运算的
# 运算之前将张量扩充为同维矩阵 再进行运算
a = torch.arange(3).reshape((3, 1))
b = torch.arange(2).reshape((1, 2))
print(a,b) # 分别为3行1列 一行2列
           # 进行运算时会同时扩充为3行2列(复制型扩充)
print(a+b) # 具体来说 矩阵a将复制列，矩阵b将复制行，然后再按元素相加

tensor([[0],
        [1],
        [2]]) tensor([[0, 1]])
tensor([[0, 1],
        [1, 2],
        [2, 3]])


## 索引和切片
### 注： py里的区间表示的是左闭右开  -1表示最后一个元素 -2 表示倒数第二个 以此类推

In [62]:
X,X[1:3],X[1:],X[:1],X[0:3,2:4]

(tensor([[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.]]),
 tensor([[ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.]]),
 tensor([[ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.]]),
 tensor([[0., 1., 2., 3.]]),
 tensor([[ 2.,  3.],
         [ 6.,  7.],
         [10., 11.]]))

## 节省内存
例如执行 $$ Y = Y + X $$ 
Python首先会计算Y + X，为结果分配新的内存，然后使Y指向内存中的这个新位置，因此并不会原地运行,会产生新的开辟内存空间消耗
> 这可能是不可取的，原因有两个：首先，我们不想总是不必要地分配内存。在机器学习中，我们可能有数百兆的参数，并且在一秒内多次更新所有参数。通常情况下，我们希望原地执行这些更新。其次，我们可能通过多个变量指向相同参数。如果我们不原地更新，其他引用仍然会指向旧的内存位置，这样我们的某些代码可能会无意中引用旧的参数。


In [63]:
# 验证是否原地操作
before = id(Y) # id会显示Y的地址
Y = Y + X
id(Y) == before # 不相等 说明开辟了新的内存空间 

False

In [64]:
# 解决原地操作问题 使用切片赋值
before = id(X)
X += Y # 等价于 X[:] = X+Y
id(X) == before

True

## 小结
> 深度学习存储和操作数据的主要接口是张量（ n 维数组）。它提供了各种功能，包括基本数学运算、广播、索引、切片、内存节省和转换其他Python对象

## 小练习
1. 运行本节中的代码。将本节中的条件语句X == Y更改为X < Y或X > Y，然后看看你可以得到什么样的张量。

2. 用其他形状（例如三维张量）替换广播机制中按元素操作的两个张量。结果是否与预期相同？

In [66]:
# 练习1
X,Y,X<Y,X>Y

(tensor([[ 2.,  3.,  8.,  9.],
         [ 9., 12., 15., 18.],
         [20., 21., 22., 23.]]),
 tensor([[ 2.,  2.,  6.,  6.],
         [ 5.,  7.,  9., 11.],
         [12., 12., 12., 12.]]),
 tensor([[False, False, False, False],
         [False, False, False, False],
         [False, False, False, False]]),
 tensor([[False,  True,  True,  True],
         [ True,  True,  True,  True],
         [ True,  True,  True,  True]]))

练习2 [查阅文档可知](https://pytorch.apachecn.org/docs/1.0/notes_broadcasting.html) 
## PyTorch操作支持广播，它的Tensor参数可以自动扩展为相同的类型大小(不需要复制数据）。

### 一般语义
#### 如果遵守以下规则，则两个张量是“可广播的”：

- 每个张量至少有一个维度；
- 遍历张量维度大小时，从末尾开始遍历，两个张量的维度大小要么相等,要么其中一个1或者是不存在。 全部符合即可


In [81]:
# 练习2
a1 = torch.arange(6).reshape((2,3,1))
a2 = torch.arange(6).reshape((3,2)) # 从维度的末尾开始比较
a1,a2,a1+a2 #相加时 会将a1,a2扩充为(2,3,2)维

(tensor([[[0],
          [1],
          [2]],
 
         [[3],
          [4],
          [5]]]),
 tensor([[0, 1],
         [2, 3],
         [4, 5]]),
 tensor([[[ 0,  1],
          [ 3,  4],
          [ 6,  7]],
 
         [[ 3,  4],
          [ 6,  7],
          [ 9, 10]]]))